In [9]:
import re
import json
import pandas as pd

In [3]:
data = pd.read_csv("../../data/data_cleaned_sentences_phases_2020-04-16.csv", sep='|', converters={'sentences': pd.eval})

In [5]:
data["title"] = data["title"].apply(lambda title: title if isinstance(title, str) else "") 
data["text"] = data.apply(lambda row: " ".join([sent for sent in [row["title"]] + row["sentences"]]), axis=1)

In [57]:
sample_data = data.sample(500)
sample_data

,id,date,pub_name,type,title,content,Label,sentences,text,phase
8775,8815,23 Ağustos 2018 Perşembe,karadeniz,yerel,olmaz teröristler ıç güvenli ülke ol cumhurbaş...,"sbskk yunan""dan dost olmaz teröristler ıç güve...",hate,[sbskk yunandan dost olmaz teröristler ıç güve...,olmaz teröristler ıç güvenli ülke ol cumhurbaş...,val
14976,15024,08 Temmuz 2016 Cuma,günboyu,Ulusal,isviçre de burka yasağı yürürlükte son iki gün...,isviçre'de burka yasağı yürürlükte isviçre nin...,not_hate,[isviçrede burka yasağı yürürlükte isviçre nin...,isviçre de burka yasağı yürürlükte son iki gün...,val
2417,2434,03 Ağustos 2016 Çarşamba,karadenizde ılkhaber,Yerel,akıl günleri ni yaşıyoruz,"""akıl günleri""m yaşıyoruz hikmet aksoy'ün yazı...",hate,[akıl günlerim yaşıyoruz hikmet aksoyün yazısı...,akıl günleri ni yaşıyoruz akıl günlerim yaşıyo...,train
21038,21086,17 Temmuz 2018 Salı,yeni meram gazetesi,Yerel,türk milleti destan yazdı,türk milleti destan yazdı vgr aj kamu-sen il t...,not_hate,[türk milleti destan yazdı vgr aj kamusen il t...,türk milleti destan yazdı türk milleti destan ...,train
12445,12493,05 Ağustos 2016 Cuma,yeni şafak,Ulusal,yargılama yaşını ye indirdi,"yargılama yaşını 'ye indirdi israil meclisi, d...",not_hate,[yargılama yaşını ye indirdi israil meclisi de...,yargılama yaşını ye indirdi yargılama yaşını y...,train
...,...,...,...,...,...,...,...,...,...,...
20809,20857,02 Temmuz 2018 Pazartesi,türkiye de yeniçağ,Ulusal,orhan uğuroğlu,son söz orhan uğuroğlu hayallerim gerçek olsa....,not_hate,[son söz orhan uğuroğlu hayallerim gerçek olsa...,orhan uğuroğlu son söz orhan uğuroğlu hayaller...,train
533,534,12 Şubat 2016 Cuma,konya hakimiyet,Yerel,medeni avrupanın yabani düşünce yapısı,medeni avrupanın yabani düşünce yapısı rusya'n...,hate,[medeni avrupanın yabani düşünce yapısı rusyan...,medeni avrupanın yabani düşünce yapısı medeni ...,train
10907,10951,22 Ekim 2019 Salı,güneş,ulusal,haddini bil,"çhp'liizmir tunç soyer, yunan ağzıyla konuştu....",hate,"[çhpliizmir tunç soyer yunan ağzıyla konuştu, ...",haddini bil çhpliizmir tunç soyer yunan ağzıyl...,train
7252,7285,09 Nisan 2018 Pazartesi,yörem gazetesi,yerel,zekertyü kılıç,zekertyü kılıç ölmem zanneden leşler bir taraf...,hate,[zekertyü kılıç ölmem zanneden leşler bir tara...,zekertyü kılıç zekertyü kılıç ölmem zanneden l...,train


In [60]:
class LinguisticRuleAssigner:
    def __init__(self):
        self.general_rules = [        
            r"\s[IRK](lık|lik|luk|lük)(.*?)(yap)",
            r"\s[IRK_KIMLIK](?!istan|stan)(.*?)(skandalı)",
            r"\s[KIMLIK](?!istan|stan)(.*?)(işgali)",
            r"\s[IRK_KIMLIK](?!istan|stan)(a|e)\s(bak sen)",
            r"\s[IRK](?!istan|stan)(.*?)(işgali)",
            r"\s[IRK](?!istan|stan)(.*?)\s[IRK](?!istan|stan)(lığını|liğini|luğunu|lüğünü)(.*?)(yap)",
            r"\s[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(uşağı|işbirlikçisi|piyonu|kuklası)(.*?)\s[IRK](?!istan|stan)",
            r"\s[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(hain|vahşi|insanlık dışı|hunharca|kan donduran|şeytani|sinsi|[ADJBEF])\s(teşebbüsleri|planları|oluşumları|[ADJAFTER])",
            r"\s[IRK](?!istan|stan)(.*?)(kurşunlarıyla|bombalarıyla|parasıyla)",
            r"\s[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(gerçekleştirdiği|yaptığı)(.*?)(katliam|zulüm|soykırım|[ADJAFTER])",
            r"\s[IRK](?!istan|stan)\s(tarafından)\s(gerçekleştirilen|yapılan)(.*?)(saldırılar|katliam|zulüm|soykırım|[ADJAFTER])",
            r"\s[IRK](?!istan|stan)\s(tarafından)\s[IRK](?!istan|stan)(a|e)(yönelik)?(.*?)(saldırılar|katliam|zulüm|soykırım|[ADJAFTER])",
            r"\s[IRK](?!istan|stan)\s(tarafından)(.*?)(saldırıya|katliama|zulme|soykırıma)(.*?)(maruz kal|uğra)",
            r"\s[IRK]\s(destekli)(.*?)\s[IRK](?!istan|stan)(darbesi|saldırıları|katliamı|soykırımı)",
            r"\s[IRK](?!istan|stan)(.*?)(öldürdü|katletti|etnik temizlik yaptı|kirletti|bastı|şehit etti)",
            r"\s[IRK](?!istan|stan)\s(tarafından)(.*?)(öldürüldü|katledildi|basıldı|şehit edildi)"
        ]
        self.anti_hs_rules = [
            r"""\s[IRK](?!istan|stan)\s(çeteleri|fanatikler|fanatiği|askerleri|milisleri|
                                        militanları|yerleşimciler|milliyetçiler|güçleri|
                                        isyanı|yaygaracılığı|polisi)""",
            r"(radikal|ırkçı|fanatik)\s[IRK](?!istan|stan)",
            r"\s[IRK](?!istan|stan)(cı|ci|çi|cu|cü|çü)\sterör\sörgütü"
        ]
        
        self.special_patterns = json.load(open("special_patterns.json", "r"))
        self.hs_specific_verbs = json.load(open("hs_specific_verbs.json", "r"))
        self.anti_hs_patterns = json.load(open("anti_hs.json", "r"))
        self.keywords = json.load(open("targeted_keywords.json", "r"))
        self.adj_bef_keyword = json.load(open("adjacent_before_keyword.json", "r"))
        self.adj_after_keyword = json.load(open("adjacent_after_keyword.json", "r"))
        
        
    
    def __detect_special_pattern(self, text):
        pattern_list = []
        for _, patterns in self.special_patterns.items():
            if len(patterns) != 0:
                patterns = f"\s({'|'.join(patterns)})|({'|'.join(patterns)})\s"
                patterns = r'{}'.format(patterns)
                finded_texts = re.findall(patterns, text, flags=re.I)
                if len(finded_texts) > 0:
                    pattern_list.append(1)
                else:
                    pattern_list.append(0)
            else:
                pattern_list.append(0)
        return pattern_list
    
    def __detect_general_pattern(self, text):
        pattern_list = []
        for _, patterns in self.general_patterns.items():            
            patterns = f"\s({'|'.join(patterns)})|({'|'.join(patterns)})\s"
            patterns = r'{}'.format(patterns)
            finded_texts = re.findall(patterns, text, flags=re.I)
            if len(finded_texts) > 0:
                pattern_list.append(1)
            else:
                pattern_list.append(0)
        return pattern_list
    
    def apply_rules(self, data):
        data["special_pattern"] = data["text"].apply(lambda text: self.__detect_special_pattern(text))
        return data

In [62]:
rule_assigner = LinguisticRuleAssigner()

In [63]:
sample_data =  rule_assigner.apply_rules(sample_data)

In [66]:
sample_data.sample(20)

,id,date,pub_name,type,title,content,Label,sentences,text,phase,special_pattern
8222,8262,19 Haziran 2018 Salı,yeni mesaj,ulusal,rumları batı azdırıyor,rumları batı azdırıyor başbakan yardımcısı ve ...,hate,[rumları batı azdırıyor başbakan yardımcısı ve...,rumları batı azdırıyor rumları batı azdırıyor ...,train,"[0, 0, 0, 0, 0]"
16682,16730,03 Mayıs 2017 Çarşamba,yeni şafak,Ulusal,rvp den gol sözü,f. bahçe kariyerinde g. saray ve beşiktaş'a ka...,not_hate,"[bahçe kariyerinde, saray ve beşiktaşa karşı o...",rvp den gol sözü bahçe kariyerinde saray ve be...,train,"[0, 0, 0, 0, 0]"
23961,24011,23 Eylül 2019 Pazartesi,doğru haber,ulusal,ahmet yücebağ,"batı'nın mısır'daki sisi dağılıyor suriye, yem...",not_hate,[batının mısırdaki sisi dağılıyor suriye yemen...,ahmet yücebağ batının mısırdaki sisi dağılıyor...,test,"[0, 0, 0, 0, 0]"
11205,11249,27 Aralık 2019 Cuma,milli gazete,ulusal,kim bir millete benzerse da onlardandır,ey iman cuma cmnü naınaz için caan vanildıaı h...,hate,[ey iman cuma cmnü naınaz için caan vanildıaı ...,kim bir millete benzerse da onlardandır ey ima...,train,"[0, 0, 0, 0, 0]"
3554,3577,15 Mayıs 2017 Pazartesi,konyanın sesi,yerel,mesele atatürk değil aslında,mesele atatürk değil aslında- geçen hafta perş...,hate,[mesele atatürk değil aslında geçen hafta perş...,mesele atatürk değil aslında mesele atatürk de...,test,"[0, 0, 0, 0, 0]"
6753,6785,02 Mart 2018 Cuma,bizim sivas,yerel,öğrencilerden kocalı katliamını unutma sergisi,"öğrencilerden ""kocalı katliamını unutma'' serg...",hate,[öğrencilerden kocalı katliamını unutma sergis...,öğrencilerden kocalı katliamını unutma sergisi...,train,"[0, 0, 0, 0, 0]"
18393,18441,12 Ocak 2018 Cuma,bursa sancak,Yerel,ypg li örgüt üyeleri örgütü deşifre etti,ypg'li örgüt üyeleri örgütü deşifre etti eror ...,not_hate,[ypgli örgüt üyeleri örgütü deşifre etti eror ...,ypg li örgüt üyeleri örgütü deşifre etti ypgli...,train,"[0, 0, 0, 0, 0]"
12172,12220,28 Mart 2019 Perşembe,türkiye de yeniçağ,ulusal,suriyeliler suriyelileri gasbetti,"suriyeliler, suriyelileri gasbetti adana nın y...",hate,[suriyeliler suriyelileri gasbetti adana nın y...,suriyeliler suriyelileri gasbetti suriyeliler ...,train,"[0, 0, 0, 0, 0]"
14985,15033,08 Temmuz 2016 Cuma,yeni konya,Yerel,kulu bayramlaştı,"kulu bayramlaştı konya'nın kulu ilçesinde, bay...",not_hate,[kulu bayramlaştı konyanın kulu ilçesinde bayr...,kulu bayramlaştı kulu bayramlaştı konyanın kul...,test,"[0, 0, 0, 0, 0]"
10896,10940,05 Eylül 2019 Perşembe,ısparta gazetesi,yerel,anlamayan kafalara yıl önce cumhuriyet için bö...,m. tanzer ünal anlamayan kafalara! yıl önce cu...,hate,[tanzer ünal anlamayan kafalara yıl önce cumhu...,anlamayan kafalara yıl önce cumhuriyet için bö...,train,"[0, 0, 0, 0, 0]"


In [69]:
sample_data.query("id == 11294").text.values

array(['ali aydofian ali aydofian korkak ve yalancı gazetemizin arşivine grin ve benim tüm yazılarımda batılıların sinsi planlarını iki yüzlü olduklarını hıristiyan dayanışması içinde olduklarını haçlı zihniyetinde olduklarını defalarca yazdım bu yazdıklarımın ispatı olarak da batılıların oynadıkları oyunları eylemlerini örnek vermiştim dışişleri bakanımız dünkü olayların gelişmesiyle ilgili olarak görüşme yaptığı avrupalı meslektaşlarıyla vardığı sonuç olarak açıklama yaparken maalesef avrupalılar dayanışma içindeler nedenle hollanda ve almanya olaylarını kınamaya yanaşmıyorlar mealinde konuştu işte bu kadar hiç avrupalılara kızmayın hakaret etmeyin tüm suç bizde öylesine kendimizden geçmiş kendimizi kadar batıya batılı değerlere batılı kültüre kaptırmış adapte olmuş öylesine kendimizi uyutmuşuz ki onların altımızı oyduklarının farkına bile varmamışız dostlarımız demişler bayılmışız ingilizce konuşabiliyor diye bir başbakanı gözlerimiz yaş doluyor sevinçten müttefikiz demişler neyimiz

In [12]:
special_patterns = json.load(open("special_patterns.json", "r"))

In [21]:
pattern_list = []
for degree, patterns in special_patterns.items():
    if len(patterns) != 0:
        patterns = f"\s({'|'.join(patterns)})|({'|'.join(patterns)})\s"
        patterns = r'{}'.format(patterns)
        finded_texts = re.findall(patterns, text, flags=re.I)
        if len(finded_texts) > 0:
            pattern_list.append(1)
        else:
            pattern_list.append(0)
    else:
        pattern_list.append(0)

\s(vahşi toplumlar|batı cehaleti|haçlı zihniyeti|haçlı ruhu|kriptolar)|(vahşi toplumlar|batı cehaleti|haçlı zihniyeti|haçlı ruhu|kriptolar)\s
\s(batıl batı)|(batıl batı)\s
\s(yahudi ajanı|yahudi uşağı|kripto ermeni|suriyeli işgali|afgan işgali|mülteci işgali|pakistanlı işgali|arapların işgali)|(yahudi ajanı|yahudi uşağı|kripto ermeni|suriyeli işgali|afgan işgali|mülteci işgali|pakistanlı işgali|arapların işgali)\s
\s(yahudi çakallığı|katil rum|haydut rumlar|rum zorbalığı|kafir alevi|ateist alevi|ateist aleviler|ateist PKK)|(yahudi çakallığı|katil rum|haydut rumlar|rum zorbalığı|kafir alevi|ateist alevi|ateist aleviler|ateist PKK)\s


In [17]:
patterns

['vahşi toplumlar',
 'batı cehaleti',
 'haçlı zihniyeti',
 'haçlı ruhu',
 'kriptolar']

In [18]:
'|'.join(patterns)

'vahşi toplumlar|batı cehaleti|haçlı zihniyeti|haçlı ruhu|kriptolar'

In [141]:
re.findall(r'[rum|rus|ermeni][luk|lik].*?yap', "bugün rusluk konusunda konuşma yapan insan da çok rumluk yapamayan da çok.")

['rusluk konusunda konuşma yap', 'rumluk yap']

In [183]:
re.findall(r'(rum|fransızlar|ermeniler)(un|ın|in).*(hain|barbar)+', "bu rumun haince tutumlarından ötürü ve fransızların barbar yaklaşımları ve ermenilerin bu durmaz barbar bizi yordu.", flags=re.I|re.S)

[('rum', 'un', 'barbar')]

In [202]:
re.findall(r'(rum|fransızlar|ermeniler)(un|ın|in)(.*?)(hain|barbar)(ce|ca)?', "bu rumun haince tutumlarından ötürü ve fransızların barbar yaklaşımları ve ermenilerin bu durmaz barbar bizi yordu.")

[('rum', 'un', ' ', 'hain', 'ce'),
 ('fransızlar', 'ın', ' ', 'barbar', ''),
 ('ermeniler', 'in', ' bu durmaz ', 'barbar', '')]

In [243]:
re.findall(r"(rum|ermeni|yunan)(?!istan|stan)(cı|ci|çi|cu|cü|çü)\sterör\sörgütü", "ermenici terör örgütünde değil onlar, bildiğin yunancı terör örgütü gibi saldırıyorlar")

[('ermeni', 'ci'), ('yunan', 'cı')]

In [246]:
pattern = r"\s(rum|ermeni|yunan)(?!istan|stan)(lık|lik|luk|lük)(.*?)(yap)"
re.findall(pattern, "bu caniler yorumluk da yapar ermenilik de yapar.", flags=re.I)

[('ermeni', 'lik', ' de ', 'yap')]

In [251]:
pattern = r"\s(rum|ermeni|yunan)(?!istan|stan)(.*?)\s(rum|ermeni|yunan)(?!istan|stan)(?!istan|stan)(lığını|liğini|luğunu|lüğünü)(.*?)(yap)"
re.findall(pattern, "bu böyledir beyler rum yunanlığını yapar vazgeçmez.", flags=re.I)

[('rum', '', 'yunan', 'lığını', ' ', 'yap')]

In [71]:
data[data["text"].str.contains("a bak sen")]

,id,date,pub_name,type,title,content,Label,sentences,text,phase
731,734,07 Mart 2016 Pazartesi,ayrıntılı haber,Yerel,aslanların payları tehlikeye girince,aslanların payları tehlikeye girince yirmi yıl...,hate,[aslanların payları tehlikeye girince yirmi yı...,aslanların payları tehlikeye girince aslanları...,train
5773,5801,15 Ocak 2018 Pazartesi,batı akdeniz gazetesi,yerel,chp bildiğiniz gibi,av. ramazan yıldırım chp bildiğiniz gibi değer...,hate,[ramazan yıldırım chp bildiğiniz gibi değerli ...,chp bildiğiniz gibi ramazan yıldırım chp bildi...,train
9220,9261,11 Haziran 2019 Salı,yeni dönem,yerel,rumlara bak sen,"rumlara bak sen! kıbrıs rum yönetimi, doğu akd...",hate,[rumlara bak sen kıbrıs rum yönetimi doğu akde...,rumlara bak sen rumlara bak sen kıbrıs rum yön...,train
10567,10611,07 Kasım 2019 Perşembe,hamle niğde,yerel,bende virüs var,durmuş göktekin bende virüs var! bakıyorum hay...,hate,[durmuş göktekin bende virüs var bakıyorum hay...,bende virüs var durmuş göktekin bende virüs va...,train
11250,11294,04 Ocak 2019 Cuma,gazete ipekyol,yerel,hak ehlinin hüznü,hak ehli nin huznu ceviz at l/l t. bir arkadaş...,hate,"[hak ehli nin huznu ceviz at ll, bir arkadaşım...",hak ehlinin hüznü hak ehli nin huznu ceviz at ...,train
13588,13636,24 Mayıs 2016 Salı,türkiye,Ulusal,nerde eski tatlar,"sorduk ustasına, anlattı. nenle eski tatlar ir...",not_hate,"[sorduk ustasına anlattı, nenle eski tatlar ir...",nerde eski tatlar sorduk ustasına anlattı nenl...,train
18408,18456,15 Ocak 2018 Pazartesi,yeni asır,Ulusal,bay kemalin soykırımcı başkanı,canan kaftana oğlu pntc-an '• tarihte bugün: e...,not_hate,[canan kaftana oğlu pntcan tarihte bugün ermen...,bay kemalin soykırımcı başkanı canan kaftana o...,train


In [77]:
data.query("id == 5801").text.values

array(['chp bildiğiniz gibi ramazan yıldırım chp bildiğiniz gibi değerli okurlar demokrasi ile yönetilen birden fazla siyası partilerin bulunduğu ülkelerde demokrasinin olmazsa olmazı muhalefet partileridir bir ülkede ıyı bir muhalefet partisi aynı zamanda partileri iktidarı dizginlemek iktidarın güç zehirlenmesini önlemek açısından son derece elzemdir aklı başında bir muhalefet sadece iktidarı eleştirmez aynı zamanda yol göstericidir de on beş yıllık ak parti iktidarı döneminde aklı başında bir muhalefetin olmaması ak parti için bir şans değildir eğer aklı başında bir muhalefet partisi olsa ıdı türkiyenin geldiği nokta çok daha farklı olabilirdi ülkemizin en hayatı en stratejik konularında bile iktidar ile masaya beraberce yumruk vuran bir muhalefet partisi elbette bu ülkenin çıkarları ıçm çok çok elzemdir belki chp düzelir yapıcı bir muhalefet yapar umuduna ne zaman kapılsak hep bir hayal kırıklığı yaşadık yukarıda yazdıklarımdan varmak istediğim yer sırasıdır chp nın dün istanbul il

In [236]:
general_rules = [
    r"[IRK](lık|lik|luk|lük)(.*?)(yap)",
    r"[IRK_KIMLIK](?!istan|stan)(.*?)(skandalı)",
    r"[KIMLIK](?!istan|stan)(.*?)(işgali)",
    r"[IRK_KIMLIK](?!istan|stan)(a|e)\s(bak sen)",
    r"[IRK](?!istan|stan)(.*?)(işgali)",
    r"[IRK](?!istan|stan)(.*?)[IRK](?!istan|stan)(lığını|liğini|luğunu|lüğünü)(.*?)(yap)",
    r"[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(uşağı|işbirlikçisi|piyonu|kuklası)(.*?)[IRK](?!istan|stan)",
    r"[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(hain|vahşi|insanlık dışı|hunharca|kan donduran|şeytani|sinsi|teşebbüsleri|planları|oluşumları)",
    r"[IRK](?!istan|stan)(.*?)(kurşunlarıyla|bombalarıyla|parasıyla)",
    r"[IRK](?!istan|stan)(ın|in|un|ün)(.*?)(gerçekleştirdiği|yaptığı)(.*?)(katliam|zulüm|soykırım)",
    r"[IRK](?!istan|stan)\s(tarafından)\s(gerçekleştirilen|yapılan)(.*?)(saldırılar|katliam|zulüm|soykırım)",
    r"[IRK](?!istan|stan)\s(tarafından)\s[IRK](?!istan|stan)(a|e)(yönelik)?(.*?)(saldırılar|katliam|zulüm|soykırım)",
    r"[IRK](?!istan|stan)\s(tarafından)(.*?)(saldırıya|katliama|zulme|soykırıma)(.*?)(maruz kal|uğra)",
    r"[IRK]\s(destekli)(.*?)[IRK](?!istan|stan)(darbesi|saldırıları|katliamı|soykırımı)",
    r"[IRK](?!istan|stan)(.*?)(öldürdü|katletti|etnik temizlik yaptı|kirletti|bastı|şehit etti)",
    r"[IRK](?!istan|stan)\s(tarafından)(.*?)(öldürüldü|katledildi|basıldı|şehit edildi)"  
]

In [239]:
json.load(open("general_patterns.json", "r"))

JSONDecodeError: Invalid \escape: line 6 column 41 (char 201)